In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "neuralmind/bert-base-portuguese-cased"  # Escolha um modelo em português
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


In [ ]:
from datasets import load_dataset

dataset = load_dataset("text", data_files={"train": "datasets/train.txt", "validation": "datasets/validation.txt"})

def preprocess_function(examples):
    # Separa as falas do usuário e do chatbot
    conversations = [exemplo.split(" ### ") for exemplo in examples["text"]]
    # Concatena as falas em uma única string
    text = [" ".join(conversation) for conversation in conversations]
    return tokenizer(text, truncation=True, padding="max_length")

dataset = dataset.map(preprocess_function, batched=True)

In [3]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)

trainer.train()

C:\Users\igorb\anaconda3\Lib\site-packages\transformers\training_args.py:1449: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: expected sequence of length 13 at dim 1 (got 32)

In [ ]:
trainer.save_model("./chatbot_pt")

In [ ]:
def generate_response(text):
    input_ids = tokenizer(text, return_tensors="pt").input_ids
    output = model.generate(input_ids, max_length=128, num_beams=5, no_repeat_ngram_size=2)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

while True:
    user_input = input("Usuário: ")
    response = generate_response(user_input)
    print("Chatbot:", response)

Usuário: Olá!
Chatbot: Olá! e E e e d d D E E Em EmEmEm em em m m em en en En EnEnEn Em En enEn En Em en emEm EmEn emem mmmMM M MMm m M EEED D DDD DE De DE DE de De DeDeDe De de dededeDe de d Dede DEdeDEDEde de DEDe DEDE DE,, A A a a o O O o o e o r r R R H h H HHH h h rrr H F f f g g r m h
Usuário: O que é Python ?
Chatbot: O que é Python? o OOOoo do do que que o o comthonthon,, j e e E E eEE E O O É É é é não não Não Não Só Só só só não só também também como mas mas Mas Mas Só Também Também Eu Eu Nós Nós Nos Nossossos Os Os Outros Outros Outras Outras Outra Outra outra outra outras outras diversas diversas outras demais demais outras Outras outras tantas tantas demais vezes vezes outras as as As As AS ASASASAsAAAsAs demais demas demastastastatatátáTATAtaTATOTOtotatas aqui aqui Aqui
